## Project 2. Part 1 (Scrape & Pickle)

**Use Selenium and BeautifulSoup**

In [4]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import pickle
import pandas as pd

## Define all functions (NEED TO CHANGE THE WHILE LOOK FROM n TO True !!!!)

In [22]:
#Get urls for all the pages and put them into a list
def get_url_list():
    """Records the current url and goes through the website, clicking Next as many times as there are pages.
    Returns a list of urls to be used in the get_htmls function."""
    #Creates a list of urls for all pages
    url_list = []
    
    #Reads the url of the page the driver is currently in and adds it into the list
    current_page_url = driver.current_url
    url_list.append(current_page_url)
    
    count_except = 0
    n = 250
    while n > 0:#True:
        try:
            #Find "Next" button and click it
            search_button = driver.find_element_by_link_text('Next')
            search_button.click()
            current_page_url = driver.current_url
            url_list.append(current_page_url)
            n -= 1
            time.sleep(3)
        except:
            #Count the exceptions
            count_except += 1
            #If the there haven't been 3 exceptions yet, sleep for a bit and then continue
            if count_except < 3:
                time.sleep(10)
            else:
                #If "Next" button isn't there anymore or an error occurs, return the list
                #driver.close()
                return url_list
    #driver.close()
    return url_list


#Get all the html files for each car ad and put it into a list
def get_htmls(url, html_list):
    """Takes an url (of one page) and an existing html list.
    Returns updated html list with all the htmls from provided url."""
    #Convert the url into html and then soup it up
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html5lib')
    
    #Find all announcement items
    all_ads = soup.find_all('a', class_ = 'announcement-item')
    
    #Copy the existing list of htmls
    final_html_list = html_list
    
    for ad in all_ads:
        html = ad.get('href')
        final_html_list.append(html)
    return final_html_list

#Makes the soup from all of the html files
def make_soup(html_list, soup_list):
    """Generates and return a soup object based on a given html list"""
    final_soup_list = soup_list
    for html in html_list:
        source_code = requests.get(html)
        soup = BeautifulSoup(source_code.text, 'html5lib')
        final_soup_list.append(soup)
    return final_soup_list

#Get the make and engine size for each car
def get_make_and_engine(soup):
    """Given an input soup object retrieves the advert title (containing Make_Model and Engine size)
    Returns a dictionary with Make_Model and Engine keys"""
    dividers = soup.find_all('div', {'class', 'col-5 classifieds-info'})
    car = {}
    for div in dividers:
        for title in div('h1'):
            title_list = title.text.split(',')
            for i in range(3):
                if i == 0:
                    car['Make_Model'] = title_list[i]
                elif i == 1:
                    car['Engine_Size'] = title_list[i]
    return car    


#Get the rest of the features for each car
def get_other_features(soup, car_dict):
    """Retrieves remaining features of a car using input soup object
    Returns an updated cars dictionary"""
    car = car_dict
    params = soup.find_all('table', {'class', 'announcement-parameters'})
    for param in params:
        for tr in param('tr'):
            for th in tr('th'):
                col_title = th.text
                for td in tr('td'):
                    col_value = td.text
                    car[col_title] = col_value
    return car


#Scrape all the features for all the cars
def scrape(soup_list):
    """Given the soups file this function retrieves all the feature columns about each car"""
    #For tracking
    car_dict_list = []
    for soup in soup_list:
        #Retrieve the car make, model and engine size
        car = get_make_and_engine(soup)
        #Retrieve the rest of the features
        car = get_other_features(soup, car)
        car_dict_list.append(car)
    return car_dict_list


### Get url list

In [11]:
#Define the website
website = 'https://en.autoplius.lt/'

##Could also have a website list
#website_list = ['https://en.autoplius.lt/', 'https://en.autogidas.lt/']
#for web in website_list:
#    website = web
#    driver.get(website)

chromedriver = "/Applications/chromedriver"                     #  path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

#Go to the website
driver.get(website)

In [12]:
#Now filter on cars (not minivans, or trucks)
car_type_form = driver.find_element_by_id("cats_search_1")
car_type_form.send_keys("Cars")
car_type_form.send_keys(Keys.RETURN)

In [13]:
#Find "Search" button and click it
search_button = driver.find_element_by_link_text('Search')
search_button.click()

In [14]:
#Now get all the urls
url_list = get_url_list()
print(len(url_list))

https://en.autoplius.lt/ads/used-cars?page_nr=2
https://en.autoplius.lt/ads/used-cars?page_nr=3
https://en.autoplius.lt/ads/used-cars?page_nr=4
https://en.autoplius.lt/ads/used-cars?page_nr=5
https://en.autoplius.lt/ads/used-cars?page_nr=6
https://en.autoplius.lt/ads/used-cars?page_nr=7
https://en.autoplius.lt/ads/used-cars?page_nr=8
https://en.autoplius.lt/ads/used-cars?page_nr=9
https://en.autoplius.lt/ads/used-cars?page_nr=10
https://en.autoplius.lt/ads/used-cars?page_nr=11
https://en.autoplius.lt/ads/used-cars?page_nr=12
https://en.autoplius.lt/ads/used-cars?page_nr=13
https://en.autoplius.lt/ads/used-cars?page_nr=14
https://en.autoplius.lt/ads/used-cars?page_nr=15
https://en.autoplius.lt/ads/used-cars?page_nr=16
https://en.autoplius.lt/ads/used-cars?page_nr=17
https://en.autoplius.lt/ads/used-cars?page_nr=18
https://en.autoplius.lt/ads/used-cars?page_nr=19
https://en.autoplius.lt/ads/used-cars?page_nr=20
https://en.autoplius.lt/ads/used-cars?page_nr=21
https://en.autoplius.lt/ads/

In [15]:
with open('all_my_urls.pkl', 'wb') as picklefile2:
    pickle.dump(url_list, picklefile2)

In [16]:
with open('all_my_urls_145_20180716_14_30.pkl', 'rb') as picklefile2:
    url_list = pickle.load(picklefile2)

### Scrape

In [17]:
#It's to to get the htmls from each page (approx. 20 / page)
html_list = []
for url in url_list:
    get_htmls(url, html_list)

print(len(html_list))

https://en.autoplius.lt/ads/volkswagen-caravelle-2-4-l-other-1996-diesel-7433949.html
https://en.autoplius.lt/ads/bmw-520-2-0-l-wagon-2018-diesel-7160889.html
https://en.autoplius.lt/ads/toyota-avensis-2-0-l-wagon-2015-diesel-7437907.html
https://en.autoplius.lt/ads/bmw-328-saloon-sedan-2012-petrol-7413785.html
https://en.autoplius.lt/ads/volkswagen-passat-2-0-l-wagon-2006-diesel-7435035.html
https://en.autoplius.lt/ads/audi-a6-2-7-l-wagon-2008-diesel-7433535.html
https://en.autoplius.lt/ads/bmw-x5-3-0-l-suv-off-road-2015-petrol-7368817.html
https://en.autoplius.lt/ads/bmw-x5-3-0-l-suv-off-road-2007-diesel-7431073.html
https://en.autoplius.lt/ads/audi-a3-1-9-l-hatchback-2001-diesel-7440805.html
https://en.autoplius.lt/ads/saab-9-3-2-0-l-saloon-sedan-2007-petrol-gas-7439389.html
https://en.autoplius.lt/ads/mazda-3-1-6-l-hatchback-2011-diesel-7253597.html
https://en.autoplius.lt/ads/bmw-525-2-5-l-saloon-sedan-2005-diesel-7437439.html
https://en.autoplius.lt/ads/infiniti-qx56-5-6-l-suv-of

https://en.autoplius.lt/ads/ford-escort-1-8-l-commercial-2000-diesel-5482415.html
https://en.autoplius.lt/ads/mercedes-benz-250-2-5-l-saloon-sedan-1990-diesel-7363313.html
https://en.autoplius.lt/ads/volkswagen-golf-1-4-l-hatchback-1998-petrol-gas-7435657.html
https://en.autoplius.lt/ads/lada-21011-1-2-l-saloon-sedan-1986-petrol-7440041.html
https://en.autoplius.lt/ads/renault-espace-2-2-l-mpv-minivan-1998-diesel-7434411.html
https://en.autoplius.lt/ads/rover-75-2-0-l-saloon-sedan-2000-diesel-7440769.html
https://en.autoplius.lt/ads/volkswagen-golf-1-9-l-hatchback-1992-diesel-7435773.html
https://en.autoplius.lt/ads/bmw-318-1-8-l-saloon-sedan-1991-petrol-7433003.html
https://en.autoplius.lt/ads/citroen-xsara-1-4-l-hatchback-1999-petrol-7435725.html
https://en.autoplius.lt/ads/opel-corsa-1-2-l-hatchback-2001-petrol-7125257.html
https://en.autoplius.lt/ads/peugeot-406-2-0-l-saloon-sedan-1999-diesel-7432579.html
https://en.autoplius.lt/ads/audi-a4-1-9-l-saloon-sedan-1997-diesel-7121575.ht

https://en.autoplius.lt/ads/citroen-xsara-picasso-1-8-l-mpv-minivan-2000-petrol-gas-6919425.html
https://en.autoplius.lt/ads/chrysler-voyager-2-5-l-mpv-minivan-2001-diesel-7131683.html
https://en.autoplius.lt/ads/citroen-xsara-1-6-l-hatchback-2002-petrol-7432945.html
https://en.autoplius.lt/ads/toyota-avensis-2-0-l-wagon-2002-diesel-7435741.html
https://en.autoplius.lt/ads/volkswagen-golf-1-9-l-wagon-1999-diesel-7436865.html
https://en.autoplius.lt/ads/citroen-c5-2-2-l-wagon-2002-diesel-7436005.html
https://en.autoplius.lt/ads/honda-accord-2-2-l-saloon-sedan-2004-diesel-7294341.html
https://en.autoplius.lt/ads/nissan-micra-1-5-l-hatchback-2005-diesel-7438059.html
https://en.autoplius.lt/ads/opel-zafira-2-2-l-mpv-minivan-2002-diesel-7437869.html
https://en.autoplius.lt/ads/peugeot-307-1-6-l-wagon-2008-diesel-7435529.html
https://en.autoplius.lt/ads/volkswagen-sharan-1-9-l-mpv-minivan-2000-diesel-7438275.html
https://en.autoplius.lt/ads/audi-a6-2-5-l-wagon-1996-diesel-7439589.html
https:

https://en.autoplius.lt/ads/audi-a4-2-5-l-wagon-2000-diesel-7433761.html
https://en.autoplius.lt/ads/skoda-fabia-1-9-l-hatchback-2003-diesel-7440337.html
https://en.autoplius.lt/ads/ford-mondeo-2-0-l-saloon-sedan-2004-petrol-7439277.html
https://en.autoplius.lt/ads/volkswagen-golf-1-9-l-hatchback-2001-diesel-7432923.html
https://en.autoplius.lt/ads/jaguar-x-type-3-0-l-saloon-sedan-2001-petrol-7000327.html
https://en.autoplius.lt/ads/volkswagen-golf-1-9-l-wagon-2004-diesel-7439617.html
https://en.autoplius.lt/ads/audi-a6-2-5-l-wagon-2003-diesel-7437315.html
https://en.autoplius.lt/ads/skoda-octavia-1-9-l-wagon-2006-diesel-7434835.html
https://en.autoplius.lt/ads/volkswagen-golf-1-9-l-hatchback-2003-diesel-7431693.html
https://en.autoplius.lt/ads/volvo-v70-2-4-l-hatchback-2002-petrol-gas-7435077.html
https://en.autoplius.lt/ads/audi-a6-2-5-l-wagon-2000-diesel-7438273.html
https://en.autoplius.lt/ads/ford-fiesta-1-4-l-hatchback-2003-diesel-7432633.html
https://en.autoplius.lt/ads/citroen-

https://en.autoplius.lt/ads/mercedes-benz-vito-2-3-l-mpv-minivan-1998-diesel-7439587.html
https://en.autoplius.lt/ads/toyota-corolla-verso-1-6-l-mpv-minivan-2005-petrol-7439563.html
https://en.autoplius.lt/ads/volvo-s40-2-4-l-saloon-sedan-2005-petrol-7425435.html
https://en.autoplius.lt/ads/volkswagen-jetta-1-9-l-saloon-sedan-2006-diesel-7440671.html
https://en.autoplius.lt/ads/opel-astra-1-7-l-saloon-sedan-2008-diesel-7119839.html
https://en.autoplius.lt/ads/nissan-tiida-1-8-l-hatchback-2007-petrol-gas-7262273.html
https://en.autoplius.lt/ads/opel-astra-1-7-l-wagon-2008-diesel-7434223.html
https://en.autoplius.lt/ads/honda-accord-2-0-l-saloon-sedan-2003-petrol-7435401.html
https://en.autoplius.lt/ads/volkswagen-golf-1-6-l-hatchback-2005-petrol-7440835.html
https://en.autoplius.lt/ads/skoda-fabia-1-9-l-wagon-2008-diesel-7439507.html
https://en.autoplius.lt/ads/rover-75-2-5-l-wagon-2003-petrol-gas-7152075.html
https://en.autoplius.lt/ads/audi-a8-2-5-l-saloon-sedan-2002-diesel-7162713.ht

...

https://en.autoplius.lt/ads/mazda-626-2-0-l-hatchback-1998-petrol-7434871.html
https://en.autoplius.lt/ads/wartburg-353-1-3-l-saloon-sedan-1990-petrol-7069819.html
https://en.autoplius.lt/ads/audi-80-2-0-l-saloon-sedan-1992-petrol-7346361.html
https://en.autoplius.lt/ads/peugeot-406-2-0-l-saloon-sedan-1999-petrol-7435131.html
https://en.autoplius.lt/ads/peugeot-307-1-6-l-hatchback-2003-petrol-7215113.html
https://en.autoplius.lt/ads/audi-100-2-8-l-saloon-sedan-1992-petrol-gas-7425249.html
https://en.autoplius.lt/ads/saab-9-3-2-2-l-hatchback-2001-diesel-6351805.html
https://en.autoplius.lt/ads/lancia-lybra-wagon-2000-petrol-gas-7367487.html
https://en.autoplius.lt/ads/mazda-626-1-8-l-saloon-sedan-1998-petrol-7366747.html
https://en.autoplius.lt/ads/renault-scenic-1-9-l-mpv-minivan-1999-diesel-7342823.html
https://en.autoplius.lt/ads/audi-a4-1-6-l-saloon-sedan-1995-petrol-7399923.html
https://en.autoplius.lt/ads/nissan-almera-2-2-l-hatchback-2001-diesel-5698481.html
https://en.autoplius.

https://en.autoplius.lt/ads/seat-ibiza-1-9-l-hatchback-2001-diesel-7342837.html
https://en.autoplius.lt/ads/volkswagen-golf-1-6-l-hatchback-1989-petrol-7187655.html
https://en.autoplius.lt/ads/audi-a4-1-6-l-saloon-sedan-1995-petrol-7297631.html
https://en.autoplius.lt/ads/opel-astra-1-6-l-saloon-sedan-1992-petrol-7320181.html
https://en.autoplius.lt/ads/ford-mondeo-2-0-l-saloon-sedan-2002-diesel-7184599.html
https://en.autoplius.lt/ads/mercedes-benz-c200-2-0-l-saloon-sedan-1994-diesel-6565647.html
https://en.autoplius.lt/ads/volkswagen-golf-1-9-l-hatchback-1998-diesel-7402647.html
https://en.autoplius.lt/ads/opel-astra-1-7-l-wagon-1999-diesel-7426537.html
https://en.autoplius.lt/ads/volkswagen-golf-1-6-l-hatchback-1998-petrol-7417975.html
https://en.autoplius.lt/ads/audi-a3-1-6-l-hatchback-1997-petrol-7405421.html
https://en.autoplius.lt/ads/jaguar-x-type-2-0-l-saloon-sedan-2007-diesel-6837057.html
https://en.autoplius.lt/ads/ford-focus-1-6-l-hatchback-2008-petrol-6322833.html
https://

https://en.autoplius.lt/ads/honda-prelude-2-0-l-coupe-1999-petrol-gas-5832215.html
https://en.autoplius.lt/ads/chevrolet-venture-3-4-l-mpv-minivan-1998-petrol-7016317.html
https://en.autoplius.lt/ads/seat-toledo-1-9-l-saloon-sedan-2001-diesel-7315545.html
https://en.autoplius.lt/ads/nissan-almera-tino-2-2-l-hatchback-2001-diesel-7367401.html
https://en.autoplius.lt/ads/mercedes-benz-e200-2-0-l-saloon-sedan-2001-petrol-7045253.html
https://en.autoplius.lt/ads/mitsubishi-carisma-saloon-sedan-1999-petrol-7421239.html
https://en.autoplius.lt/ads/volkswagen-passat-1-6-l-wagon-1997-petrol-7421875.html
https://en.autoplius.lt/ads/volkswagen-transporter-2-4-l-commercial-1993-diesel-7430053.html
https://en.autoplius.lt/ads/volkswagen-passat-2-8-l-saloon-sedan-2000-petrol-7435559.html
https://en.autoplius.lt/ads/ssangyong-musso-2-9-l-pick-up-2004-diesel-7382297.html
https://en.autoplius.lt/ads/mercedes-benz-a160-1-6-l-hatchback-2003-petrol-6787825.html
https://en.autoplius.lt/ads/toyota-avensis-

https://en.autoplius.lt/ads/saab-9-5-2-3-l-saloon-sedan-2002-petrol-gas-7328653.html
https://en.autoplius.lt/ads/citroen-xsara-picasso-1-8-l-mpv-minivan-2000-petrol-7330361.html
https://en.autoplius.lt/ads/opel-astra-1-8-l-hatchback-1999-petrol-7393979.html
https://en.autoplius.lt/ads/nissan-primera-1-6-l-wagon-2005-petrol-7394461.html
https://en.autoplius.lt/ads/chrysler-town-country-3-3-l-mpv-minivan-2003-petrol-gas-7315233.html
https://en.autoplius.lt/ads/mercedes-benz-c180-1-8-l-saloon-sedan-1996-petrol-7398505.html
https://en.autoplius.lt/ads/volkswagen-golf-1-9-l-wagon-1996-diesel-7298933.html
https://en.autoplius.lt/ads/mazda-323f-1-6-l-hatchback-1991-petrol-6622681.html
https://en.autoplius.lt/ads/nissan-primera-2-0-l-saloon-sedan-2001-petrol-6021141.html
https://en.autoplius.lt/ads/volkswagen-sharan-1-9-l-mpv-minivan-1997-diesel-7427347.html
https://en.autoplius.lt/ads/opel-vectra-saloon-sedan-1999-diesel-7353267.html
https://en.autoplius.lt/ads/opel-manta-1-8-l-coupe-1978-pet

https://en.autoplius.lt/ads/peugeot-307-2-0-l-hatchback-2001-diesel-7334731.html
https://en.autoplius.lt/ads/vauxhall-kita-1-8-l-other-2004-petrol-7419913.html
https://en.autoplius.lt/ads/volkswagen-golf-1-4-l-hatchback-1998-petrol-7373317.html
https://en.autoplius.lt/ads/volkswagen-passat-1-8-l-saloon-sedan-1997-petrol-7333405.html
https://en.autoplius.lt/ads/renault-scenic-1-9-l-mpv-minivan-2000-diesel-7429865.html
https://en.autoplius.lt/ads/volvo-s40-1-8-l-saloon-sedan-1998-petrol-5928597.html
https://en.autoplius.lt/ads/skoda-fabia-1-9-l-wagon-2002-diesel-7406251.html
https://en.autoplius.lt/ads/renault-grand-espace-2-0-l-mpv-minivan-1999-petrol-7335625.html
https://en.autoplius.lt/ads/renault-laguna-2-0-l-wagon-2002-petrol-7127677.html
https://en.autoplius.lt/ads/ford-focus-1-8-l-hatchback-2000-diesel-7440819.html
https://en.autoplius.lt/ads/audi-100-2-2-l-saloon-sedan-1987-petrol-7274405.html
https://en.autoplius.lt/ads/volvo-v70-2-4-l-wagon-1998-petrol-7215137.html
https://en.a

In [23]:
#Let's make soup (yuck!)
soup_list = []
soup_list = make_soup(html_list, soup_list)
print(len(soup_list))

2900


In [26]:
#Let's pickle the soup just in case
with open('pickled_soup_20180716_14_30.pkl', 'wb') as picklefile3:
    pickle.dump(soup_list, picklefile3)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [25]:
#Now we can unpickle and continue to work with it
with open('pickled_soup_20180716_14_30.pkl', 'wb') as picklefile3:
    soup_list = pickle.load(picklefile3)

UnsupportedOperation: read

In [27]:
#Scrape the hell out of this website!!!
cars_dict = scrape(soup_list)
len(cars_dict)

{'Make_Model': 'Volkswagen Caravelle', 'Engine_Size': ' 2.4 l.', 'Price in Lithuania': '2 550 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Date of manufacture': '1996-07', 'Mileage': '450 000 km', 'Engine': '2400 cm³, 74 HP (55kW)', 'Fuel type': 'Diesel', 'Body type': 'Other', 'Number of doors': '4/5', 'Driven wheels': 'Front wheel drive (FWD)', 'Gearbox': 'Manual', 'Color': 'White', 'Damage': 'No damages', 'Steering wheel': 'Left hand drive (LHD)', 'MOT test expiry': '2019-08', 'Wheel size': 'R15', 'Number of seats': '7', 'VIN check': '\n        \n            Check vehicle history\n                    \n    ', 'Security': 'Central locking,Alarm', 'Audio/video equipment': 'CD player,MP3 player,Sound amplifier,Subwoofer', 'Exterior': 'Light alloy rims,Tow hook', 'Interior': 'Tinted windows,Leather seats,Vairo stiprintuvas,Auxiliary heating', 'Safety': 'ABS,Airbags', 'Tuning (improvements)': 'Tuned interior'}
{'Make_Model': 'BMW 520', 'Engine_Size': ' 2.0 l.', 'Price in Lithuania

{'Make_Model': 'Audi A4', 'Engine_Size': ' 3.0 l.', 'Price in Lithuania': '5 200 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Export price': '5 200 €', 'Date of manufacture': '2007-08', 'Mileage': '256 000 km', 'Engine': '3000 cm³, 232 HP (171kW)', 'Fuel type': 'Diesel', 'Body type': 'Wagon', 'Number of doors': '4/5', 'Driven wheels': 'All wheel (4х4)', 'Gearbox': 'Manual', 'Climate control': 'Climate control', 'Color': 'Gray / silver', 'Damage': 'No damages', 'Steering wheel': 'Left hand drive (LHD)', 'Wheel size': 'R16', 'Number of seats': '5', 'Euro standard': 'Euro 4', 'First registration country': 'Sweden', 'VIN number': 'WAUZZZ8E48A027948', 'VIN check': '\n        \n            Check vehicle history\n                            \n                    \n    ', 'Audio/video equipment': 'Navigation/GPS', 'Exterior': 'Light alloy rims,Metallic paint,Tow hook', 'Electronics': 'Electric windows,Cruise control', 'Interior': 'Sport seats,Tinted windows,Multifunctional steering whe

{'Make_Model': 'Mercedes-Benz E270', 'Engine_Size': ' 2.7 l.', 'Price in Lithuania': '3 200 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Date of manufacture': '2002-08', 'Mileage': '278 000 km', 'Engine': '2685 cm³, 163 HP (120kW)', 'Fuel type': 'Diesel', 'Body type': 'Saloon / sedan', 'Number of doors': '4/5', 'Driven wheels': 'Rear wheel drive (RWD)', 'Gearbox': 'Automatic', 'Climate control': 'Climate control', 'Color': 'Gray / silver', 'Damage': 'No damages', 'Steering wheel': 'Left hand drive (LHD)', 'MOT test expiry': '2020-06', 'Number of seats': '5', 'First registration country': 'France', 'VIN check': '\n        \n            Check vehicle history\n                    \n    ', 'Security': 'Central locking', 'Audio/video equipment': 'CD player,CD changer,AUX input,Navigation/GPS,Original audio system', 'Exterior': 'Light alloy rims,Metallic paint,Fog lights,Xenon lights', 'Electronics': 'Electric mirrors,Electric windows,Automatic headlamps,Outside temperature sensor,On

{'Make_Model': 'BMW 535', 'Engine_Size': ' 3.0 l.', 'Price in Lithuania': '5 950 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Date of manufacture': '2004-10', 'Mileage': '300 000 km', 'Engine': '3000 cm³, 271 HP (200kW)', 'Fuel type': 'Diesel', 'Body type': 'Saloon / sedan', 'Number of doors': '4/5', 'Driven wheels': 'Rear wheel drive (RWD)', 'Gearbox': 'Automatic', 'Climate control': 'Climate control', 'Color': 'Black', 'Damage': 'No damages', 'Steering wheel': 'Left hand drive (LHD)', 'MOT test expiry': '2019-09', 'Wheel size': 'R18', 'Number of seats': '5', 'First registration country': 'Austria', 'VIN check': '\n        \n            Check vehicle history\n                    \n    ', 'Security': 'Central locking,Alarm', 'Audio/video equipment': 'CD player,MP3 player,CD changer,AUX input,Navigation/GPS,Original audio system,Handsfree kit', 'Exterior': 'Light alloy rims,Metallic paint,Fog lights,Xenon lights,Headlights washers', 'Electronics': 'Electric mirrors,In-car mobile

{'Make_Model': 'BMW X5', 'Engine_Size': ' 3.0 l.', 'Price in Lithuania': '18 750 €', 'Date of manufacture': '2010-12', 'Mileage': '120 000 km', 'Engine': '2993 cm³, 305 HP (225kW)', 'Fuel type': 'Diesel', 'Body type': 'SUV / off-road', 'Number of doors': '4/5', 'Driven wheels': 'All wheel (4х4)', 'Gearbox': 'Automatic', 'Climate control': 'Climate control', 'Color': 'Black', 'Damage': 'No damages', 'Steering wheel': 'Left hand drive (LHD)', 'Wheel size': 'R19', 'Number of seats': '5', 'Euro standard': 'Euro 5', 'First registration country': 'Italy', 'VIN check': '\n        \n            Check vehicle history\n                    \n    ', 'Security': 'Central locking,Immobilizer,Alarm', 'Audio/video equipment': 'CD player,Navigation/GPS,Original audio system,USB input', 'Exterior': 'Light alloy rims,Fog lights,Xenon lights,Headlights washers', 'Electronics': 'Electric mirrors,Galinio vaizdo kamera,In-car mobile phone,Electric windows,Automatic headlamps,Outside temperature sensor,On-boa

...

{'Make_Model': 'Volkswagen Passat', 'Engine_Size': ' 1.8 l.', 'Price in Lithuania': '800 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Date of manufacture': '1998', 'Engine': '1800 cm³, 125 HP (92kW)', 'Fuel type': 'Petrol / LPG', 'Body type': 'Wagon', 'Number of doors': '4/5', 'Driven wheels': 'Front wheel drive (FWD)', 'Gearbox': 'Manual', 'Color': 'Red / crimson', 'Damage': 'No damages', 'Steering wheel': 'Left hand drive (LHD)', 'MOT test expiry': '2019-04', 'Wheel size': 'R15', 'Number of seats': '5', 'VIN check': '\n        \n            Check vehicle history\n                    \n    '}
{'Make_Model': 'Seat Arosa', 'Engine_Size': ' 1.7 l.', 'Price in Lithuania': '800 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Export price': '800 €', 'Date of manufacture': '1999', 'Mileage': '240 000 km', 'Engine': '1716 cm³, 59 HP (44kW)', 'Fuel type': 'Diesel', 'Body type': 'Hatchback', 'Number of doors': '2/3', 'Driven wheels': 'Front wheel drive (FWD)', 'Gearbox': 'Manual', 

{'Make_Model': 'Volvo V40', 'Engine_Size': ' 1.8 l.', 'Price in Lithuania': '800 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Export price': '800 €', 'Date of manufacture': '2000-01', 'Engine': '1783 cm³, 122 HP (90kW)', 'Fuel type': 'Petrol', 'Body type': 'Wagon', 'Number of doors': '4/5', 'Driven wheels': 'Front wheel drive (FWD)', 'Gearbox': 'Manual', 'Climate control': 'Air conditioning', 'Color': 'Green / olive', 'Damage': 'No damages', 'Steering wheel': 'Left hand drive (LHD)', 'Wheel size': 'R16', 'Kerb weight, kg': '1335', 'Number of seats': '5', 'VIN check': '\n        \n            Check vehicle history\n                    \n    ', 'Urban': '8.00', 'Extra-urban': '6.80', 'Combined': '7.00', 'Exterior': 'Light alloy rims,Metallic paint', 'Electronics': 'Electric windows', 'Interior': 'Vairo stiprintuvas', 'Safety': 'Airbags'}
{'Make_Model': 'Audi A6', 'Engine_Size': ' 2.5 l.', 'Price in Lithuania': '800 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Date of manu

{'Make_Model': 'Ford Focus', 'Engine_Size': ' 1.8 l.', 'Price in Lithuania': '800 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Export price': '800 €', 'Date of manufacture': '1999-04', 'Mileage': '260 000 km', 'Engine': '1753 cm³, 89 HP (66kW)', 'Fuel type': 'Diesel', 'Body type': 'Wagon', 'Number of doors': '4/5', 'Driven wheels': 'Front wheel drive (FWD)', 'Gearbox': 'Manual', 'Color': 'Gray / silver', 'Damage': 'No damages', 'Steering wheel': 'Left hand drive (LHD)', 'MOT test expiry': '2019-03', 'Wheel size': 'R14', 'Kerb weight, kg': '1032', 'Number of seats': '5', 'VIN check': '\n        \n            Check vehicle history\n                    \n    ', 'Urban': '6.60', 'Extra-urban': '4.40', 'Combined': '4.90', 'Exterior': 'Tow hook', 'Interior': 'Vairo stiprintuvas', 'Other features': 'Service book'}
{'Make_Model': 'Volvo S70', 'Engine_Size': ' 2.5 l.', 'Price in Lithuania': '800 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Date of manufacture': '1998', 'Engine':

{'Make_Model': 'Ford Focus', 'Engine_Size': ' 1.8 l.', 'Price in Lithuania': '829 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Date of manufacture': '2000-01', 'Engine': '1800 cm³, 89 HP (66kW)', 'Fuel type': 'Diesel', 'Body type': 'Hatchback', 'Number of doors': '4/5', 'Gearbox': 'Manual', 'Damage': 'No damages', 'Steering wheel': 'Left hand drive (LHD)', 'MOT test expiry': '2020-02', 'VIN check': '\n        \n            Check vehicle history\n                    \n    '}
{'Make_Model': 'Audi 100', 'Engine_Size': ' 2.2 l.', 'Price in Lithuania': '830 €\n    \n        Sužinokite įmoką\n         »\n\n', 'Export price': '830 €', 'Date of manufacture': '1987-07', 'Mileage': '239 245 km', 'Engine': '2226 cm³, 137 HP (101kW)', 'Fuel type': 'Petrol', 'Body type': 'Saloon / sedan', 'Number of doors': '4/5', 'Driven wheels': 'Front wheel drive (FWD)', 'Gearbox': 'Manual', 'Color': 'Gray / silver', 'Damage': 'No damages', 'Steering wheel': 'Left hand drive (LHD)', 'MOT test expiry': '2

2900

**Explore the soups**

In [ ]:
cars_dict

### Create my dataset (pandas dataframe)

In [ ]:
#Now fill out the rows
car_ad_dataset = pd.DataFrame(cars_dict)
car_ad_dataset.info()

### Pickle

In [ ]:
#Let's pickle the hell out of the result (don't want to have to run this again)
with open('car_ad_dataset_20180716.pkl', 'wb') as picklefile:
    pickle.dump(car_ad_dataset, picklefile)

In [ ]:
# with open('car_ad_dataset.pkl', 'rb') as picklefile:
#     my_car_ad_dataset = pickle.load(picklefile)
    
#print(my_car_ad_dataset[:10])

## Trial and Error

**Try to optimize the first part of the scraping**

In [ ]:
#Get the make and engine size for each car
#def get_make_and_engine(soup):
#    """Given an input soup object retrieves the advert title (containing Make_Model and Engine size)
#    Returns a dictionary with Make_Model and Engine keys"""
soup = soup[20]

dividers = soup.find_all('div', {'class', 'col-5 classifieds-info'})   #potentially try with 'divider thin'
car = {}
# for div in dividers:
#     for title in div('h1'):
#         title_list = title.text.split(',')
#         for i in range(3):
#             if i == 0:
#                 car['Make_Model'] = title_list[i]
#             elif i == 1:
#                 car['Engine_Size'] = title_list[i]
                
                

for title in dividers('h1'):
    title_list = title.text.split(',')
    for i in range(3):
        if i == 0:
            car['Make_Model'] = title_list[i]
        elif i == 1:
            car['Engine_Size'] = title_list[i]
                

**Try to optimize the second part of the scraping**

In [ ]:
#Get the rest of the features for each car
#def get_other_features(soup, car_dict):
#    """Retrieves remaining features of a car using input soup object
#    Returns an updated cars dictionary"""
car_test = []
params = soup.find_all('table', {'class', 'announcement-parameters'})
for param in params:
    #for tr in param('tr'):
    for th in param('th'):
        col_title = th.text
    for td in param('td'):
        col_value = td.text
        car_test[col_title] = col_value